In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import networkx as nx
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LeakyReLU, ReLU
from keras.utils import np_utils,  to_categorical
from keras import optimizers

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.decomposition import PCA, TruncatedSVD

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/iupui-link-prediction/train_edges.csv
/kaggle/input/iupui-link-prediction/features.csv
/kaggle/input/iupui-link-prediction/sample_submission.csv


In [2]:
train_file = "/kaggle/input/iupui-link-prediction/train_edges.csv";
sample_sub = "/kaggle/input/iupui-link-prediction/sample_submission.csv"
feat_file = "/kaggle/input/iupui-link-prediction/features.csv";

In [3]:
def calc_index(G):
    
    s = list(G.nodes);
    num_nodes = max(s)+1;
    
    #Store it into a matrix for self preservation
    #num_nodes = G.number_of_nodes();
    common_neigh = np.zeros((num_nodes,num_nodes));
    salton = common_neigh.copy();
    jaccard = common_neigh.copy();
    sorensen = common_neigh.copy();
    hub_promoted = common_neigh.copy();
    hub_depressed = common_neigh.copy();
    leicht = common_neigh.copy();
    pref = common_neigh.copy();
    adamic = common_neigh.copy();
    resource = common_neigh.copy();
    
    for k in s:
        i = s.index(k);
        if (i % 100 == 0):
            print(i)
        for l in s:
            j = s.index(l);
            if (i != j):
                deg_i = G.degree[i];
                deg_j = G.degree[j];
                min_deg = deg_i;
                max_deg = deg_j;
                if (deg_j < min_deg):
                    min_deg = deg_j;
                    max_deg = deg_i;

                com = sorted(nx.common_neighbors(G,i,j));
                num_com = len(com);
                common_neigh[i,j] = num_com;
                preff = deg_i * deg_j;
                
                ada = 0;
                res = 0;
                
                for k in com:
                    ada += 1 / math.log(G.degree[k]);
                    res += 1 / (G.degree[k]);
                    
                pref[i,j] = preff;
                adamic[i,j] = ada;
                resource[i,j] = res;

                if (num_com > 0):
                    salt = num_com / math.sqrt(deg_i * deg_j);
                    jac = num_com / (deg_i + deg_j - num_com);
                    sor = 2 * num_com / (deg_i + deg_j);
                    hub_d = num_com / min_deg;
                    hub_p = num_com / max_deg;
                    lec = num_com / (deg_i * deg_j);

                    salton[i,j] = salt;
                    jaccard[i,j] = jac;
                    sorensen[i,j] = sor;
                    hub_promoted[i,j] = hub_p;
                    hub_depressed[i,j] = hub_d;
                    leicht[i,j] = lec;
                
    
    return(common_neigh,salton,jaccard,sorensen,hub_promoted,hub_depressed,leicht,pref,adamic,resource);

def calc_index_nodes(k,l,s,G):
    
    i = s.index(k);
    j = s.index(l);
    
    deg_i = G.degree[i];
    deg_j = G.degree[j];
    min_deg = deg_i;
    max_deg = deg_j;
    if (deg_j < min_deg):
        min_deg = deg_j;
        max_deg = deg_i;

    com = sorted(nx.common_neighbors(G,i,j));
    num_com = len(com);
    pref = deg_i * deg_j;

    ada = 0;
    res = 0;

    for k in com:
        ada += 1 / math.log(G.degree[k]);
        res += 1 / (G.degree[k]);

    salt = 0;
    jac = 0;
    sor = 0;
    hub_d = 0;
    hub_p = 0;
    lec = 0;
        
    if (num_com > 0):
        salt = num_com / math.sqrt(deg_i * deg_j);
        jac = num_com / (deg_i + deg_j - num_com);
        sor = 2 * num_com / (deg_i + deg_j);
        hub_d = num_com / min_deg;
        hub_p = num_com / max_deg;
        lec = num_com / (deg_i * deg_j);                
    
    return(num_com,salt,jac,sor,hub_p,hub_d,lec,pref,ada,res);

def calc_common_neigh(G):
    #Store it into a matrix for self preservation
    num_nodes = G.number_of_nodes();
    common_neigh = np.zeros((num_nodes,num_nodes));
    
    for i in range(num_nodes):
        if (i % 100 == 0):
            print(i)
        for j in range(num_nodes):
            if (i != j):
                num_com = len(sorted(nx.common_neighbors(G,i,j)));
                common_neigh[i,j] = num_com
    
    return(common_neigh);

In [4]:
#Load the input files
import csv

G = nx.Graph()

with open(train_file) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1]=='1':
            edge= row[0].split('-')
            #print(int(edge[0]),int(edge[1]))
            G.add_edge(int(edge[0]),int(edge[1]))
            
#s = sorted(G.nodes);
s = G.nodes
s = list(s)

adj = nx.adjacency_matrix(G);
print(adj[146,1356])
print(adj[s.index(146),s.index(1356)])

0
1


In [5]:
# next let's read in the possible edges we need to classify from the test file
#FUCK THE WAY MOHLER DID THIS
#It assumes all nodes are connected so fuck
#But we will let it stand...

Gsub = nx.Graph()

with open(sample_sub) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1]=='1' or row[1]=='0':
            edge= row[0].split('-')
            #print(edge[0],edge[1]);
            Gsub.add_edge(int(edge[0]),int(edge[1]))
            
G.add_nodes_from(Gsub.nodes)
print(G.number_of_nodes());
s = list(G.nodes);

2708


In [6]:
#Select only the training data
train_df = pd.read_csv(train_file);
train_df = train_df.values;
sz = np.shape(train_df);
train_data = np.zeros((sz[0],3));

for i in range(sz[0]):
    edge = train_df[i,0];
    edge = edge.split('-');
    train_data[i,0] = int(edge[0])
    train_data[i,1] = int(edge[1]);
    train_data[i,2] = int(train_df[i,1]);
    


In [7]:
#Load the feature file
feat = pd.read_csv(feat_file,header=None);
feat = feat.values;
print(feat)
print(feat.shape)

#feat = feat.sort(axis=0);
#print(feat)

def feat_find(x,feat):
    y = feat[feat[:,0]==x][0][1:1433];
    return y;

y = feat_find(1911,feat)
print(np.sum(y))
print(np.sum(feat[0,:]))
y = feat_find(2002,feat)
print(np.sum(y))
print(np.sum(feat[1,:]))

[[ 462    0    0 ...    0    0    0]
 [1911    0    0 ...    0    0    0]
 [2002    0    0 ...    0    0    0]
 ...
 [2372    0    0 ...    0    0    0]
 [ 955    0    0 ...    0    0    0]
 [ 376    0    0 ...    0    0    0]]
(2708, 1434)
17
482
22
1928


In [8]:
#Run K-means on the raw data
km_k_1 = 25;
km_k_2 = 10;
feat_data = feat[:,1:1433];
feat_ind = feat[:,0];
feat_ind = feat_ind.tolist()

km_1 = KMeans(n_clusters=km_k_1, random_state=0).fit(feat_data)
km_1_lab = km_1.labels_;
km_2 = KMeans(n_clusters=km_k_2, random_state=0).fit(feat_data)
km_2_lab = km_2.labels_;

print(km_1.labels_)


[24 18 18 ... 14 23  2]


In [9]:
pca = PCA(n_components = 15);
feat_pca = pca.fit_transform(feat_data)
print(pca.singular_values_)
print(np.shape(feat_pca))
#print(feat_pca)

[28.61206924 27.26813732 25.87107944 25.15121366 23.54645545 21.39462921
 19.90420152 19.31085509 18.89314444 18.3319261  18.04349181 17.80966185
 17.01715495 16.85405204 16.71814399]
(2708, 15)


In [10]:
svd = TruncatedSVD(n_components = 85, n_iter=20, random_state=42);
feat_svd = svd.fit_transform(feat_data)
print(svd.singular_values_)
print(np.shape(feat_svd))

[57.98869193 28.06106211 26.38354815 25.24221479 23.83808358 22.14033389
 21.06337249 19.41407071 18.98746452 18.35584903 18.10183269 17.88867761
 17.6055499  17.10510647 16.93320787 16.80631596 16.6921565  16.30243008
 15.89373125 15.82896717 15.76646632 15.55468424 15.40369103 15.28760085
 15.16219174 15.04766382 14.89578657 14.71406277 14.53643533 14.48211861
 14.41763954 14.21069345 14.09894907 14.01399209 13.96777078 13.8923858
 13.79994404 13.71466344 13.64534233 13.54503971 13.52705845 13.4570214
 13.32382803 13.21392131 13.17450335 13.14832187 13.09845302 13.00476802
 12.90182898 12.81273372 12.77422095 12.70430809 12.59631492 12.56330314
 12.52468224 12.4473607  12.41800849 12.36871239 12.21073195 12.15752586
 12.13269288 12.06438148 12.02922357 11.93615278 11.85921974 11.8480262
 11.81961144 11.77329098 11.69113592 11.64075408 11.61774024 11.60468142
 11.49157561 11.43416323 11.38265364 11.35332112 11.29960403 11.27701037
 11.20746452 11.1285104  11.11567122 11.08802154 11.04

In [11]:
#Now I need to make this into something that I can feed into a neural net like nothing
num_nodes = max(s)+1;
adj = nx.adjacency_matrix(G);
adj = adj.todense();

sz_train = np.shape(train_df);
print(sz_train[0])
sz_pca = np.shape(feat_pca);
sz_svd = np.shape(feat_svd);
datum = np.zeros((sz_train[0],24+km_k_1*2+km_k_2*2+sz_pca[1]*2+sz_svd[1]*2));
datum_1 = datum.copy();
ind = 0;

print(sz_train[0])

eps = .0001;

for ind in range(sz_train[0]):
    
    if (ind % 1000 == 0):
        print(ind)

    k = train_data[ind,0];
    l = train_data[ind,1];
    i = s.index(k);
    j = s.index(l);
    
    com, salt, jac, sor, hub_p, hub_d, lec, pref, res, ada = calc_index_nodes(k,l,s,G);
    
    datum[ind,0] = com;
    datum[ind,1] = salt;
    datum[ind,2] = jac;
    datum[ind,3] = sor;
    datum[ind,4] = hub_p;
    datum[ind,5] = hub_d;
    datum[ind,6] = lec;
    datum[ind,7] = pref;
    datum[ind,8] = res;
    datum[ind,9] = ada;

    deg_i = G.degree[i];
    deg_j = G.degree[j];
    min_deg = deg_i;
    max_deg = deg_j;
    if (min_deg > max_deg):
        max_deg = deg_j;
        min_deg = deg_i;
    datum[ind,10] = min_deg;
    datum[ind,11] = max_deg;

    #print(ind,k,l)
    y1 = feat_find(k,feat);
    y2 = feat_find(l,feat);

    dp = np.dot(y1,y2);
    n1 = np.linalg.norm(y1);
    n2 = np.linalg.norm(y2);
    dp_norm = dp / (n1*n2);

    c_ang = math.acos(dp_norm);

    datum[ind,12] = dp;
    datum[ind,13] = dp_norm;
    datum[ind,14] = c_ang;

    #Covariance stuff
    #Use the cov and inv cov

    #Covariance of the features
    c_cov = np.cov(y1,y2);
    new_cov = c_cov.copy();

    for i in range(np.shape(new_cov)[0]):
        new_cov[i,i] += eps

    new_cov = np.linalg.inv(new_cov);

    c_cov = c_cov.flatten();
    new_cov = new_cov.flatten();

    datum[ind,15:19] = c_cov;
    datum[ind,19:23] = new_cov;
    
    #Kmeans
    i_f_ind = feat_ind.index(k);
    j_f_ind = feat_ind.index(l);
    
    k_c1 = km_1_lab[i_f_ind];
    k_c2 = km_1_lab[j_f_ind];
    
    
    datum_1[ind,:23] = datum[ind,:23];
    
    datum[ind,23+k_c1] = 1;
    datum[ind,23+km_k_1+k_c2] = 1;
    datum_1[ind,23+k_c2] = 1;
    datum_1[ind,23+km_k_1+k_c1] = 1;
    
    
    k_c1 = km_2_lab[i_f_ind];
    k_c2 = km_2_lab[j_f_ind];
    datum[ind,23+km_k_1*2+k_c1] = 1;
    datum[ind,23+km_k_1*2+km_k_2+k_c2] = 1;
    datum_1[ind,23+k_c2] = 1;
    datum_1[ind,23+km_k_1*2+km_k_2+k_c1] = 1;

    #PCA
    p_c1 = feat_pca[i_f_ind,:];
    p_c2 = feat_pca[j_f_ind,:];
    new_st = 23+km_k_1*2+km_k_2*2;
    
    datum[ind,new_st:new_st+sz_pca[1]] = p_c1;
    datum[ind,new_st+sz_pca[1]:new_st+sz_pca[1]*2] = p_c2;
    datum_1[ind,new_st:new_st+sz_pca[1]] = p_c2;
    datum_1[ind,new_st+sz_pca[1]:new_st+sz_pca[1]*2] = p_c1;
    
    #SVD
    s_c1 = feat_svd[i_f_ind,:];
    s_c2 = feat_svd[j_f_ind,:];
    #print(s_c1)
    new_st = 23+km_k_1*2+km_k_2*2+sz_pca[1]*2;
    #print(new_st,new_st+sz_svd[1])
    
    datum[ind,new_st:new_st+sz_svd[1]] = s_c1;
    datum[ind,new_st+sz_svd[1]:new_st+sz_svd[1]*2] = s_c2;
    datum_1[ind,new_st:new_st+sz_svd[1]] = s_c2;
    datum_1[ind,new_st+sz_svd[1]:new_st+sz_svd[1]*2] = s_c1;
    
    
    datum[ind,-1] = train_data[ind,2];
    datum_1[ind,-1] = train_data[ind,2];
        

#Combine our datums
datum = np.concatenate((datum,datum_1),axis=0);    
    
print(np.shape(datum))
print(datum)

7388
7388
0
1000
2000
3000
4000
5000
6000
7000
(14776, 294)
[[ 0.          0.          0.         ...  0.27500408 -0.50795508
   0.        ]
 [ 2.          0.10166571  0.01538462 ... -0.30014728  0.23763707
   1.        ]
 [ 5.          0.30429031  0.14705882 ...  0.27261364 -0.10893412
   1.        ]
 ...
 [ 0.          0.          0.         ... -0.02024884 -0.05162364
   1.        ]
 [ 0.          0.          0.         ...  0.10292158  0.07433107
   0.        ]
 [ 0.          0.          0.         ...  0.20397512 -0.45180778
   0.        ]]


In [12]:
noisey_datum = datum.copy();
sz_dat = np.shape(datum);
num_entry = sz_dat[0] * (sz_dat[1]-1);
noisey = np.random.normal(1,.03,num_entry);
noisey = noisey.reshape(-1,sz_dat[1]-1);
noisey_datum[:,:-1] *= noisey;
#print(noisey_datum)
datum = np.concatenate((datum,noisey_datum),axis=0); 
print(np.shape(datum));
print(datum);

(29552, 294)
[[ 0.          0.          0.         ...  0.27500408 -0.50795508
   0.        ]
 [ 2.          0.10166571  0.01538462 ... -0.30014728  0.23763707
   1.        ]
 [ 5.          0.30429031  0.14705882 ...  0.27261364 -0.10893412
   1.        ]
 ...
 [ 0.          0.          0.         ... -0.01971337 -0.04987134
   1.        ]
 [ 0.          0.          0.         ...  0.10569838  0.07462603
   0.        ]
 [ 0.          0.          0.         ...  0.20288791 -0.46614556
   0.        ]]


In [13]:
#Get test data
test_df = pd.read_csv(sample_sub);
test_df = test_df.values;
test_sz = np.shape(test_df);
test_inp = np.zeros((test_sz[0],3));

for i in range(test_sz[0]):
    edge = test_df[i,0];
    edge = edge.split('-');
    test_inp[i,0] = int(edge[0])
    test_inp[i,1] = int(edge[1]);
    test_inp[i,2] = int(test_inp[i,1]);
    
#print(train_data)

num_nodes = max(s)+1;

test_datum = np.zeros((test_sz[0],23+km_k_1*2+km_k_2*2+sz_pca[1]*2+sz_svd[1]*2));
test_datum_1 = test_datum.copy();
ind = 0;

for ind in range(test_sz[0]):
    
    if (ind % 1000 == 0):
        print(ind)

    k = int(test_inp[ind,0]);
    l = int(test_inp[ind,1]);
    i = s.index(k);
    j = s.index(l);
    
    com, salt, jac, sor, hub_p, hub_d, lec, pref, res, ada = calc_index_nodes(k,l,s,G);
    
    test_datum[ind,0] = com;
    test_datum[ind,1] = salt;
    test_datum[ind,2] = jac;
    test_datum[ind,3] = sor;
    test_datum[ind,4] = hub_p;
    test_datum[ind,5] = hub_d;
    test_datum[ind,6] = lec;
    test_datum[ind,7] = pref;
    test_datum[ind,8] = res;
    test_datum[ind,9] = ada;

    deg_i = G.degree[i];
    deg_j = G.degree[j];
    min_deg = deg_i;
    max_deg = deg_j;
    if (min_deg > max_deg):
        max_deg = deg_j;
        min_deg = deg_i;
    
    test_datum[ind,10] = min_deg;
    test_datum[ind,11] = max_deg;
    
    y1 = feat_find(k,feat);
    y2 = feat_find(l,feat);

    dp = np.dot(y1,y2);
    n1 = np.linalg.norm(y1);
    n2 = np.linalg.norm(y2);
    dp_norm = dp / (n1*n2);

    c_ang = math.acos(dp_norm);

    test_datum[ind,12] = dp;
    test_datum[ind,13] = dp_norm;
    test_datum[ind,14] = c_ang;
    
    #Covariance of the features
    c_cov = np.cov(y1,y2);
    new_cov = c_cov.copy();

    for i in range(np.shape(new_cov)[0]):
        new_cov[i,i] += eps

    new_cov = np.linalg.inv(new_cov);

    c_cov = c_cov.flatten();
    new_cov = new_cov.flatten();

    test_datum[ind,15:19] = c_cov;
    test_datum[ind,19:23] = new_cov;
    
    
    #Kmeans
    i_f_ind = feat_ind.index(k);
    j_f_ind = feat_ind.index(l);
    
    k_c1 = km_1_lab[i_f_ind];
    k_c2 = km_1_lab[j_f_ind];
    
    test_datum_1[ind,:23] = test_datum[ind,:23];
    
    test_datum[ind,23+k_c1] = 1;
    test_datum[ind,23+km_k_1+k_c2] = 1;
    
    test_datum_1[ind,23+k_c2] = 1;
    test_datum_1[ind,23+km_k_1+k_c1] = 1;
    
    k_c1 = km_2_lab[i_f_ind];
    k_c2 = km_2_lab[j_f_ind];
    
    test_datum[ind,23+km_k_1*2+k_c1] = 1;
    test_datum[ind,23+km_k_1*2+km_k_2+k_c2] = 1;
    
    test_datum_1[ind,23+k_c2] = 1;
    test_datum_1[ind,23+km_k_1*2+km_k_2+k_c1] = 1;
    
    #PCA
    p_c1 = feat_pca[i_f_ind,:];
    p_c2 = feat_pca[j_f_ind,:];
    new_st = 23+km_k_1*2+km_k_2*2;
    
    test_datum[ind,new_st:new_st+sz_pca[1]] = p_c1;
    test_datum[ind,new_st+sz_pca[1]:new_st+sz_pca[1]*2] = p_c2;
    test_datum_1[ind,new_st:new_st+sz_pca[1]] = p_c2;
    test_datum_1[ind,new_st+sz_pca[1]:new_st+sz_pca[1]*2] = p_c1;
    
    #SVD
    s_c1 = feat_svd[i_f_ind,:];
    s_c2 = feat_svd[j_f_ind,:];
    #print(s_c1)
    new_st = 23+km_k_1*2+km_k_2*2+sz_pca[1]*2;
    #print(new_st,new_st+sz_svd[1])
    
    test_datum[ind,new_st:new_st+sz_svd[1]] = s_c1;
    test_datum[ind,new_st+sz_svd[1]:new_st+sz_svd[1]*2] = s_c2;
    test_datum_1[ind,new_st:new_st+sz_svd[1]] = s_c2;
    test_datum_1[ind,new_st+sz_svd[1]:new_st+sz_svd[1]*2] = s_c1;


print(test_datum)

0
1000
2000
3000
[[ 0.          0.          0.         ...  0.2661687  -0.01524116
   0.02393302]
 [ 0.          0.          0.         ...  0.21524577  0.00919083
  -0.03208832]
 [ 0.          0.          0.         ... -0.02950152 -0.02547247
  -0.12677891]
 ...
 [ 0.          0.          0.         ... -0.01306529  0.01784848
  -0.04614251]
 [ 0.          0.          0.         ... -0.09300129 -0.00743396
   0.32274806]
 [ 0.          0.          0.         ...  0.0078797   0.17753173
   0.08620801]]


In [14]:
train_df = pd.DataFrame(datum);
train_df.to_csv("train.csv",header=None,index=None)
test_df = pd.DataFrame(test_datum);
test_df.to_csv("test.csv",header=None,index=None)

In [15]:
#Normalization
max_col = np.amax(datum,axis=0);
datum /= max_col;
test_datum /= max_col[:-1];
test_datum_1 /= max_col[:-1];

In [16]:
#I now have the training and testing data
#Separate the training data into train and test
Y = datum[:,-1];
X = datum[:,:-1];

sz_X = np.shape(X);

#"""
X = X.reshape(-1,sz_X[1],1);
test_datum = test_datum.reshape(-1,sz_X[1],1);
test_datum_1 = test_datum_1.reshape(-1,sz_X[1],1);
#"""

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = .25);

In [17]:
#"""
#create convolution neural network
model = Sequential()
model.add(Dense(512, input_shape=(sz_X[1],1)))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Flatten())

model.add(Dense(256))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.25))

model.add(Dense(1, activation='sigmoid'))

model.summary();
#"""

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 293, 512)          1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 293, 512)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 293, 512)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 150016)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               38404352  
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)              

In [18]:
#"""
#Compile model
sgd = optimizers.SGD(lr=.01);
model.compile(loss='MSE', 
              optimizer='adam', 
              metrics=['accuracy'])

history = model.fit(X_train,y_train, shuffle=True,
          batch_size=25,epochs=150,verbose=1,
          validation_data=(X_test,y_test))

Y_out_0 = model.predict(test_datum)
Y_out_1 = model.predict(test_datum_1);

Y_out = ( Y_out_0 + Y_out_1 ) / 2;

print(Y_out)
#"""

Train on 22164 samples, validate on 7388 samples
Epoch 1/150
22164/22164 [==============================] - 18s 792us/step - loss: 0.1908 - accuracy: 0.7237 - val_loss: 0.1741 - val_accuracy: 0.7474
Epoch 2/150
22164/22164 [==============================] - 16s 706us/step - loss: 0.1723 - accuracy: 0.7554 - val_loss: 0.1555 - val_accuracy: 0.7814
Epoch 3/150
22164/22164 [==============================] - 16s 720us/step - loss: 0.1571 - accuracy: 0.7801 - val_loss: 0.1488 - val_accuracy: 0.7952
Epoch 4/150
22164/22164 [==============================] - 16s 713us/step - loss: 0.1423 - accuracy: 0.8016 - val_loss: 0.1406 - val_accuracy: 0.8131
Epoch 5/150
22164/22164 [==============================] - 16s 700us/step - loss: 0.1307 - accuracy: 0.8207 - val_loss: 0.1345 - val_accuracy: 0.8228
Epoch 6/150
22164/22164 [==============================] - 16s 711us/step - loss: 0.1208 - accuracy: 0.8373 - val_loss: 0.1270 - val_accuracy: 0.8265
Epoch 7/150
22164/22164 [==========================

In [19]:
Y_out_0 = model.predict(test_datum)
Y_out_1 = model.predict(test_datum_1);

Y_out = ( Y_out_0 + Y_out_1 ) / 2;

print(Y_out)

[[0.        ]
 [0.99996734]
 [0.        ]
 ...
 [1.        ]
 [1.        ]
 [0.        ]]


In [20]:
"""
max_num = 0;
max_acc = 0;
#Create and train the random forest classifier
for i in [20,30,40,50,60,70,80]:
#for i in [5]:
    rf_model = RandomForestClassifier(max_depth=i,n_estimators=2500);
    rf_model.fit(X_train,y_train);
    Y_pred = rf_model.predict(X_test);
    a = accuracy_score(y_test,Y_pred);
    print(i,a);
    if (a > max_acc):
        max_acc = a;
        max_num = i;

#We choose XXXX RF since best accuracy
#print(np.mean(Y_pred))
#print(np.mean(y_test))
#We choose max_depth of 5 as our model
rf_model = RandomForestClassifier(max_depth=95,n_estimators=10000);
rf_model.fit(X_train,y_train);
Y_pred = rf_model.predict(X_test);
a = accuracy_score(y_test,Y_pred);
print(i,a);

Y_out_0 = rf_model.predict_proba(test_datum)
Y_out_1 = rf_model.predict_proba(test_datum_1);

Y_out = ( Y_out_0 + Y_out_1 ) / 2;
Y_out = Y_out[:,1]

print(Y_out)
print(np.mean(Y_out))
#"""

'\nmax_num = 0;\nmax_acc = 0;\n#Create and train the random forest classifier\nfor i in [20,30,40,50,60,70,80]:\n#for i in [5]:\n    rf_model = RandomForestClassifier(max_depth=i,n_estimators=2500);\n    rf_model.fit(X_train,y_train);\n    Y_pred = rf_model.predict(X_test);\n    a = accuracy_score(y_test,Y_pred);\n    print(i,a);\n    if (a > max_acc):\n        max_acc = a;\n        max_num = i;\n\n#We choose XXXX RF since best accuracy\n#print(np.mean(Y_pred))\n#print(np.mean(y_test))\n#We choose max_depth of 5 as our model\nrf_model = RandomForestClassifier(max_depth=95,n_estimators=10000);\nrf_model.fit(X_train,y_train);\nY_pred = rf_model.predict(X_test);\na = accuracy_score(y_test,Y_pred);\nprint(i,a);\n\nY_out_0 = rf_model.predict_proba(test_datum)\nY_out_1 = rf_model.predict_proba(test_datum_1);\n\nY_out = ( Y_out_0 + Y_out_1 ) / 2;\nY_out = Y_out[:,1]\n\nprint(Y_out)\nprint(np.mean(Y_out))\n#'

In [21]:
"""
#Create an SVM

#Create a simple one
lin_svc = svm.SVC(kernel='linear');
lin_svc.fit(X_train,y_train);
Y_lin = lin_svc.predict(X_test);
print(accuracy_score(Y_lin,y_test))

#RBF Kernel
rbf_svc = svm.SVC(kernel='rbf');
rbf_svc.fit(X_train,y_train);
Y_rbf = rbf_svc.predict(X_test);
print(accuracy_score(Y_rbf,y_test))

Y_lin_1 = lin_svc.predict_proba(test_datum);
Y_lin_2 = lin_svc.predict_proba(test_datum_1);
Y_out = (Y_lin_1 + Y_lin_2) / 2;
#Y_rbf = rbf_svc.predict(test_datum);
#Y_out = Y_lin.copy();
#"""

"\n#Create an SVM\n\n#Create a simple one\nlin_svc = svm.SVC(kernel='linear');\nlin_svc.fit(X_train,y_train);\nY_lin = lin_svc.predict(X_test);\nprint(accuracy_score(Y_lin,y_test))\n\n#RBF Kernel\nrbf_svc = svm.SVC(kernel='rbf');\nrbf_svc.fit(X_train,y_train);\nY_rbf = rbf_svc.predict(X_test);\nprint(accuracy_score(Y_rbf,y_test))\n\nY_lin_1 = lin_svc.predict_proba(test_datum);\nY_lin_2 = lin_svc.predict_proba(test_datum_1);\nY_out = (Y_lin_1 + Y_lin_2) / 2;\n#Y_rbf = rbf_svc.predict(test_datum);\n#Y_out = Y_lin.copy();\n#"

In [22]:
#Output outputs for stacking
y0_df = pd.DataFrame(Y_out);
y1_df = pd.DataFrame(Y_out_1);
y0_df.to_csv("Y0.csv",header=None,index=None);
y1_df.to_csv("Y1.csv",header=None,index=None);

In [23]:
#"""
# here is a simple algorithm to get you started...for each possible edge i-j in the test set
# we will count the number of nodes in the training graph they have in common...if they have
# one or more "mutual friends" then we will connect them (this will score about .67 acc on kaggle)


cutoff = .485;

alfa = 0;
with open('/kaggle/working/sub.csv', 'w') as csvfile:
    fieldnames = ['edge', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    with open(sample_sub) as csvfile2:
        reader = csv.reader(csvfile2, delimiter=',')
        for row in reader:
            if row[1]=='1' or row[1]=='0':
                edge= row[0].split('-')

                i=int(edge[0])
                j=int(edge[1])
                
                # here networkx has a common_neighbors function
                num_com_neigh=len(sorted(nx.common_neighbors(G, i, j)))
                y=0
                #if num_com_neigh>0:
                #    y=1

                edg_out=str(i)+"-"+str(j) 
                if (Y_out[alfa] > cutoff):
                    y = 1
                alfa += 1;
                writer.writerow({'edge': edg_out, 'label': y})
                #print(edg_out, num_com_neigh, y)
#"""

In [24]:
#print(np.mean(Y_out > .45))